In [7]:
import random
import time
import sys
from collections import defaultdict
from queue import PriorityQueue
from heapq import heappop, heappush


In [8]:
# Function to calculate the memory usage of an object
def get_memory_usage(obj):
    return sys.getsizeof(obj)
# Function to calculate the memory usage of the program
def get_program_memory_usage():
    frame = sys._getframe(1)
    locals_size = sum(get_memory_usage(v) for v in frame.f_locals.values())
    globals_size = sum(get_memory_usage(v) for v in frame.f_globals.values())
    total_size = locals_size + globals_size
    return total_size / (1024 ** 2)

In [9]:
# Class to represent the state of the chessboard
class BoardState:
    def __init__(self, queens, path_cost):
        self.queens = queens
        self.size = len(queens)
        self.path_cost = path_cost

# Method to generate successor states
    def generate_successors(self):
        successors = []
        for column in range(self.size):
            for row in range(self.size):
                if self.queens[column] != row:
                    successor = list(self.queens)
                    successor[column] = row
                    successors.append(BoardState(successor, self.path_cost + 1))
        return successors
 # Custom comparison method for priority queue ordering
    def __lt__(self, other):
        return self.path_cost < other.path_cost
# Function to solve the N-Queens problem using Uniform-Cost Search (UCS)
def ucs_n_queens(size):
    start_state = BoardState(random.sample(range(size), size), 0)
    priority_queue = PriorityQueue()
    priority_queue.put((0, start_state))  # Start with cost 0
    print(start_state.queens)
    while not priority_queue.empty():
        _, current_state = priority_queue.get()
        if is_goal_state(current_state.queens):
            return current_state.queens

        for successor in current_state.generate_successors():
            priority_queue.put((successor.path_cost, successor))

    return None
    # Function to check if the goal state is reached (no queens attacking each other)
def is_goal_state(state):
    # Check if the goal state is reached (no queens attacking each other)
    for i in range(len(state)):
        for j in range(i + 1, len(state)):
            if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                return False
    return True

In [10]:
# Function to calculate the heuristic value (number of conflicting queens)
def calculate_heuristic(state):
    n = len(state)
    conflicts = 0

    for i in range(n):
        for j in range(i + 1, n):
            if state[i] == state[j] or abs(state[i] - state[j]) == j - i:
                conflicts += 1

    return conflicts
# Function to generate random initial state
def generate_initial_state(n):
    state = list(range(n))
    random.shuffle(state)
    return state
# Function to get the successors of a state by moving one queen at a time
def get_successors(state):
    successors = []
    n = len(state)

    for i in range(n):
        for j in range(n):
            if state[i] != j:
                successor = list(state)
                successor[i] = j
                successors.append(successor)

    return successors
# A* algorithm to solve the N-Queens problem
def solve_n_queens(n):
    initial_state = generate_initial_state(n)
    heap = [(calculate_heuristic(initial_state), initial_state)]  # Priority queue
    visited = set()
    print(initial_state)
    while heap:
        _, state = heappop(heap)  # Get the state with the minimum heuristic value

        if calculate_heuristic(state) == 0:  # Solution found
            return state

        visited.add(tuple(state))

        successors = get_successors(state)
        for successor in successors:
            if tuple(successor) not in visited:
                priority = calculate_heuristic(successor) + len(state) + 1  # A* priority function
                heappush(heap, (priority, successor))

    return None  # No solution found

In [11]:
# Class to represent a chromosome in the Genetic Algorithm
class Chromosome:
    def __init__(self, genes):
        self.genes = genes
        self.conflicts = 0
# Class to represent a population in the Genetic Algorithm
class Population:
    def __init__(self, size, n, mutation_rate):
        self.size = size
        self.n = n
        self.mutation_rate = mutation_rate
        self.chromosomes = []
# Method to initialize the population
    def initialize(self):

        genes = random.sample(range(self.n), self.n)
        print(genes)
        chromosome = Chromosome(genes)
        self.calculate_conflicts(chromosome)
        self.chromosomes.append(chromosome)
   # Method to calculate the number of conflicts for a chromosome
    def calculate_conflicts(self, chromosome):
        conflicts = 0
        queens = chromosome.genes
        diagonal1 = defaultdict(int)
        diagonal2 = defaultdict(int)

        for i in range(self.n):
            for j in range(i + 1, self.n):
                if queens[i] == queens[j] or abs(queens[i] - queens[j]) == abs(i - j):
                    conflicts += 1

                if queens[i] - queens[j] == i - j:
                    diagonal1[i - j] += 1

                if queens[i] + queens[j] == i + j:
                    diagonal2[i + j] += 1

        conflicts += sum(val * (val - 1) // 2 for val in diagonal1.values())
        conflicts += sum(val * (val - 1) // 2 for val in diagonal2.values())

        chromosome.conflicts = conflicts
   # Method for selection in the Genetic Algorithm
    def selection(self):
        self.chromosomes = sorted(self.chromosomes, key=lambda x: x.conflicts)
        return self.chromosomes[:self.size // 2]
 # Method for crossover in the Genetic Algorithm
    def crossover(self, parent1, parent2):
        crossover_point = random.randint(1, self.n - 1)
        child_genes = parent1.genes[:crossover_point] + parent2.genes[crossover_point:]
        child = Chromosome(child_genes)
        self.calculate_conflicts(child)
        return child
    # Method for mutation in the Genetic Algorithm
    def mutation(self, chromosome):
        for i in range(self.n):
            if random.random() < self.mutation_rate:
                chromosome.genes[i] = random.randint(0, self.n - 1)

        self.calculate_conflicts(chromosome)
    # Method to evolve the population in the Genetic Algorithm
    def evolve(self):
        selected_chromosomes = self.selection()
        new_generation = []

        while len(new_generation) < self.size:
            parent1 = random.choice(selected_chromosomes)
            parent2 = random.choice(selected_chromosomes)
            child = self.crossover(parent1, parent2)
            self.mutation(child)
            new_generation.append(child)

        self.chromosomes = new_generation
    # Method to solve the N-Queens problem using the Genetic Algorithm
    def solve(self):
        generation = 1
        while generation <= max_generations and self.chromosomes[0].conflicts > 0:
            self.evolve()
            generation += 1
 # Method to get the best solution from the population
    def get_best_solution(self):
        return self.chromosomes[0].genes
    # Function to print the board representation
def print_board(solution):
    if solution is None:
        print("No solution found.")
        return

    n = len(solution)

    for row in range(n):
        line = ""
        for col in range(n):
            if solution[row] == col:
                line += "Q "
            else:
                line += "* "
        print(line)

In [12]:

n = int(input("Enter the number of queens: "))
algorithm = int(input("Enter the algorithm (1: Uniform-Cost Search, 2: A*, 3: Genetic Algorithm): "))
start_time = time.time()
if algorithm == 1:
    solution = ucs_n_queens(n)
elif algorithm == 2:
    solution = solve_n_queens(n)
elif algorithm == 3:
    population_size = 100
    mutation_rate = 0.1
    max_generations = 1000


    population = Population(population_size, n, mutation_rate)
    population.initialize()

    population.solve()
    solution = population.get_best_solution()
else:
    print("Invalid algorithm choice.")
    sys.exit(1)
print("Solution for N-Queens problem with N =", n)
print_board(solution)
end_time = time.time()
running_time = end_time - start_time
print("Running time: {} ms".format(running_time * 1000))
print("Memory usage of the program:", get_program_memory_usage(), "MB")

Enter the number of queens: 8
Enter the algorithm (1: Uniform-Cost Search, 2: A*, 3: Genetic Algorithm): 2
[4, 2, 7, 5, 6, 1, 3, 0]
Solution for N-Queens problem with N = 8
* * * * Q * * * 
* * Q * * * * * 
Q * * * * * * * 
* * * * * Q * * 
* * * * * * * Q 
* Q * * * * * * 
* * * Q * * * * 
* * * * * * Q * 
Running time: 11.004447937011719 ms
Memory usage of the program: 0.062259674072265625 MB
